# Upcoming Trip Stops, Part 2
Objectives:
- build a list of unique stops that are available from the next earliest trips that pass through the current stop

## Next: find a way to speed up the .isin operation perhaps with a join or a merge or something else....

---
## Import Modules & Data

In [1]:
import time
import numpy as np
import pandas as pd
import numexpr as ne
from datetime import timedelta as td

In [2]:
%load_ext line_profiler

In [3]:
stops = pd.read_feather('data/model/stops.ftr')
schedule = pd.read_feather('data/model/schedule.ftr')

In [4]:
schedule.head(1)

,trip_id,route_short_name,route_long_name,shape_id,trip_headsign,stop_sequence,stop_time,stop_time_delta,stop_id,stop_code,stop_name,stop_lat,stop_lon
0,42990004,10,VAN HORNE,886387,EAST - 10 VAN HORNE towards VICTORIA PARK,1,7:00:29,0 days 07:00:29,14155,14633,Don Mills Station,43.776222,-79.347048


In [5]:
stops.head(1)

,stop_id,stop_code,stop_name,stop_lat,stop_lon
0,262,662,Danforth Rd at Kennedy Rd,43.714379,-79.260939


In [6]:
fmt_df = schedule.copy()
fmt_df = fmt_df.sort_values(by = ['stop_time_delta', 'trip_id', 'stop_sequence'])
fmt_df['stop_time_sec'] = fmt_df.stop_time_delta.apply(lambda x: int(x.total_seconds()))
fmt_df = fmt_df.drop(columns = ['route_short_name', 'route_long_name', 'trip_headsign', 'stop_code', 'stop_name', 'stop_time', 'stop_time_delta'])

In [82]:
test_df = fmt_df.copy()
test_df

,trip_id,shape_id,stop_sequence,stop_id,stop_lat,stop_lon,stop_time_sec
450018,43003469,887148,1,8533,43.705192,-79.504228,12725
450019,43003469,887148,2,14963,43.706567,-79.504667,12755
450020,43003469,887148,3,9271,43.709123,-79.505308,12812
450021,43003469,887148,4,5572,43.713764,-79.506522,12916
450022,43003469,887148,5,1126,43.715125,-79.506944,12947
...,...,...,...,...,...,...,...
365690,43001368,886893,85,4425,43.670847,-79.295932,110408
365691,43001368,886893,86,15135,43.671493,-79.292818,110454
365692,43001368,886893,87,10371,43.672555,-79.287687,110530
365693,43001368,886893,88,4598,43.673035,-79.285343,110565


In [83]:
current_stop_id = 917
current_time_delta = td(hours = 19)
current_time_sec = int(current_time_delta.total_seconds())

In [84]:
stop_id_values = test_df.stop_id.values
stop_time_sec_values = test_df.stop_time_sec.values
select_condition = ne.evaluate('(stop_id_values == current_stop_id) & (stop_time_sec_values > current_time_sec)')

In [85]:
select_condition

array([False, False, False, ..., False, False, False])

In [96]:
x = test_df[select_condition].trip_id

In [134]:
x.index

Int64Index([1186125, 1254879, 1186055, 1188336, 1254905, 1186016, 1188266,
            1189444, 1188196, 1189405, 1188126, 1189366, 1188056, 1189296,
            1187986, 1189235, 1187916, 1189174, 1187846, 1189113, 1187776,
            1189052, 1187706, 1188991, 1187636, 1188930, 1187566, 1188869,
            1187496, 1188808, 1187426, 1188747, 1188686, 1188476, 1188616,
            1188546, 1193048, 1192978, 1192908, 1192838, 1192768, 1192698,
            1192628, 1192558, 1192488, 1192418, 1192348, 1192278, 1192208,
            1193468, 1193398, 1193328, 1193258, 1193188, 1193118,  533545,
             533395,  533320,  533470],
           dtype='int64')

In [137]:
test_df[test_df.trip_id.isin(x.to_list())]

,trip_id,shape_id,stop_sequence,stop_id,stop_lat,stop_lon,stop_time_sec
1186078,43027160,888274,1,10122,43.794687,-79.154799,66280
1186079,43027160,888274,2,3892,43.793969,-79.157635,66306
1186080,43027160,888274,3,4388,43.792623,-79.163292,66356
1254859,43029300,888390,1,14923,43.784391,-79.184989,66358
1186081,43027160,888274,4,7788,43.791840,-79.166565,66388
...,...,...,...,...,...,...,...
533488,43005603,887271,71,1852,43.744926,-79.399283,102797
533489,43005603,887271,72,9572,43.744470,-79.401416,102817
533490,43005603,887271,73,6062,43.744461,-79.404964,102850
533491,43005603,887271,74,919,43.744417,-79.405968,102859


In [138]:
%timeit test_df[test_df.trip_id.isin(test_df[select_condition].trip_id.to_list())]

16.4 ms ± 561 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [117]:
y = pd.Series(data = x, name = 'trip_id')

In [118]:
y

1186125    43027160
1254879    43029300
1186055    43027159
1188336    43027200
1254905    43029301
1186016    43027158
1188266    43027199
1189444    43027218
1188196    43027198
1189405    43027217
1188126    43027197
1189366    43027216
1188056    43027196
1189296    43027215
1187986    43027195
1189235    43027214
1187916    43027194
1189174    43027213
1187846    43027193
1189113    43027212
1187776    43027192
1189052    43027211
1187706    43027191
1188991    43027210
1187636    43027190
1188930    43027209
1187566    43027189
1188869    43027208
1187496    43027188
1188808    43027207
1187426    43027187
1188747    43027206
1188686    43027205
1188476    43027202
1188616    43027204
1188546    43027203
1193048    43027284
1192978    43027283
1192908    43027282
1192838    43027281
1192768    43027280
1192698    43027279
1192628    43027278
1192558    43027277
1192488    43027276
1192418    43027275
1192348    43027274
1192278    43027273
1192208    43027272
1193468    43027290


In [132]:
%timeit pd.merge(test_df, y)

47.1 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [130]:
test_df.join(y, on = 'trip_id', how = 'inner', rsuffix = '_l')

,trip_id,shape_id,stop_sequence,stop_id,stop_lat,stop_lon,stop_time_sec,trip_id_l


In [95]:
test_df[test_df.trip_id.isin(test_df[select_condition].trip_id)]

,trip_id,shape_id,stop_sequence,stop_id,stop_lat,stop_lon,stop_time_sec
1186078,43027160,888274,1,10122,43.794687,-79.154799,66280
1186079,43027160,888274,2,3892,43.793969,-79.157635,66306
1186080,43027160,888274,3,4388,43.792623,-79.163292,66356
1254859,43029300,888390,1,14923,43.784391,-79.184989,66358
1186081,43027160,888274,4,7788,43.791840,-79.166565,66388
...,...,...,...,...,...,...,...
533488,43005603,887271,71,1852,43.744926,-79.399283,102797
533489,43005603,887271,72,9572,43.744470,-79.401416,102817
533490,43005603,887271,73,6062,43.744461,-79.404964,102850
533491,43005603,887271,74,919,43.744417,-79.405968,102859


In [55]:
type(trip_id_series)

pandas.core.series.Series

In [61]:
trip_id_list = trip_id_series.to_list()

In [70]:
trip_index = pd.DataFrame(index = trip_id_list)

In [80]:
%timeit test_df.join(trip_index, how = 'inner', on = 'trip_id')

53.3 ms ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
test_df

,trip_id,shape_id,stop_sequence,stop_id,stop_lat,stop_lon,stop_time_sec
1186078,43027160,888274,1,10122,43.794687,-79.154799,66280
1186079,43027160,888274,2,3892,43.793969,-79.157635,66306
1186080,43027160,888274,3,4388,43.792623,-79.163292,66356
1254859,43029300,888390,1,14923,43.784391,-79.184989,66358
1186081,43027160,888274,4,7788,43.791840,-79.166565,66388
...,...,...,...,...,...,...,...
533488,43005603,887271,71,1852,43.744926,-79.399283,102797
533489,43005603,887271,72,9572,43.744470,-79.401416,102817
533490,43005603,887271,73,6062,43.744461,-79.404964,102850
533491,43005603,887271,74,919,43.744417,-79.405968,102859


In [9]:
def build_table(input_df, cur_stop_id, cur_time_sec):
    
    stop_id_values = input_df.stop_id.values
    stop_time_sec_values = input_df.stop_time_sec.values
    select_condition = ne.evaluate('(stop_id_values == cur_stop_id) & (stop_time_sec_values > cur_time_sec)')
    
    input_df = input_df[input_df.trip_id.isin(input_df[select_condition].trip_id)]

    ref_df = input_df.loc[input_df.stop_id.values == 917, ['shape_id', 'stop_sequence']].drop_duplicates().set_index('shape_id')
    input_df = input_df[input_df.stop_sequence.values > input_df.shape_id.map(ref_df.stop_sequence)]
    input_df = input_df.drop_duplicates(subset = 'stop_id')

    return input_df

In [10]:
%timeit build_table(test_df, current_stop_id, current_time_sec)

22.3 ms ± 316 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%time build_table(test_df, current_stop_id, current_time_sec)

CPU times: user 30.6 ms, sys: 1.38 ms, total: 32 ms
Wall time: 23.6 ms


,trip_id,shape_id,stop_sequence,stop_id,stop_lat,stop_lon,stop_time_sec
1186126,43027160,888274,49,5191,43.759132,-79.333893,68475
1186127,43027160,888274,50,7238,43.757624,-79.339319,68544
1186128,43027160,888274,51,5738,43.757207,-79.341181,68566
1186129,43027160,888274,52,9834,43.756053,-79.346600,68632
1186130,43027160,888274,53,6330,43.755228,-79.350256,68713
1186131,43027160,888274,54,916,43.754734,-79.352534,68765
1186132,43027160,888274,55,9974,43.753660,-79.357427,68878
1186133,43027160,888274,56,944,43.753001,-79.360766,68949
1186134,43027160,888274,57,1509,43.752639,-79.365268,69002
1186135,43027160,888274,58,9959,43.751639,-79.370302,69063


In [12]:
%lprun -f build_table build_table(test_df, current_stop_id, current_time_sec)

Timer unit: 1e-06 s

Total time: 0.032023 s
File: <ipython-input-9-c5789ec58a98>
Function: build_table at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def build_table(input_df, cur_stop_id, cur_time_sec):
     2                                               
     3         1         66.0     66.0      0.2      stop_id_values = input_df.stop_id.values
     4         1         25.0     25.0      0.1      stop_time_sec_values = input_df.stop_time_sec.values
     5         1       1332.0   1332.0      4.2      select_condition = ne.evaluate('(stop_id_values == cur_stop_id) & (stop_time_sec_values > cur_time_sec)')
     6                                               
     7         1      19773.0  19773.0     61.7      input_df = input_df[input_df.trip_id.isin(input_df[select_condition].trip_id)]
     8                                           
     9         1       3786.0   3786.0     11.8      ref_df = input_df.